About: [https://api.meteo.lt/](https://api.meteo.lt/)

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import time

Station observation data

In [25]:
vilnius_response = requests.get(url='https://api.meteo.lt/v1/stations/vilniaus-ams/observations/2023-01-01')
vilnius_data = vilnius_response.json()
print(json.dumps(vilnius_data['observations'][0], indent=2))

{
  "observationTimeUtc": "2023-01-01 00:00:00",
  "airTemperature": 7.7,
  "feelsLikeTemperature": 7.7,
  "windSpeed": 3.9,
  "windGust": 6.6,
  "windDirection": 194,
  "cloudCover": 88,
  "seaLevelPressure": 1008.1,
  "relativeHumidity": 98,
  "precipitation": 0,
  "conditionCode": "light-rain"
}


Station codes

In [2]:
response = requests.get(url='https://api.meteo.lt/v1/stations')
data = response.json()

# Extract station codes
station_codes = [station['code'] for station in data]

print(station_codes)

['birzu-ams', 'dotnuvos-ams', 'duksto-ams', 'kauno-ams', 'klaipedos-ams', 'kybartu-ams', 'laukuvos-ams', 'lazdiju-ams', 'nidos-ams', 'panevezio-ams', 'raseiniu-ams', 'siauliu-ams', 'silutes-ams', 'telsiu-ams', 'ukmerges-ams', 'utenos-ams', 'varenos-ams', 'vilniaus-ams']


Fetching temperature data from meteo.lt API

In [3]:
# set the start and end dates for data retrieval
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 31)

rate_limit = 160
delay_between_requests = 60 / rate_limit

station_codes = station_codes

url_template = 'https://api.meteo.lt/v1/stations/{station}/observations/{date}'

data_frames = []

current_date = start_date
while current_date <= end_date:
    formatted_date = current_date.strftime('%Y-%m-%d')
    
    data_list = []

    for station in station_codes:
        url = url_template.format(station=station, date=formatted_date, timeout=6000)
        
        response = requests.get(url)
        station_data = response.json()

        try:
            observations = station_data['observations']
            for observation in observations:
                observation_time_utc = observation['observationTimeUtc']
                temperature = observation['airTemperature']

                data_list.append({'Date': formatted_date, 'Station': station, 
                                  'Observation Time (UTC)': observation_time_utc,
                                  'Temperature (C)': temperature})
        except KeyError:
            print(f"No observations for {formatted_date} at station {station}")

        time.sleep(delay_between_requests)

    day_data = pd.DataFrame(data_list)
    data_frames.append(day_data)

    current_date += timedelta(days=1)

all_data = pd.concat(data_frames, ignore_index=True)

# Save the collected temperature data to a csv file

file_name = f'temperature_data_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}.csv'

all_data.to_csv(file_name, index=False)

print(f'Data saved to {file_name}')

Data saved to temperature_data_20230101_20230131.csv


Dataframe summary and information

In [4]:
print(f'shape of dataframe: {all_data.shape}, \n\nnull&NaN counts:\n{all_data.isna().sum()} \n\nfirst and last rows:\n {all_data}')

shape of dataframe: (13392, 4), 

null&NaN counts:
Date                      0
Station                   0
Observation Time (UTC)    0
Temperature (C)           0
dtype: int64 

first and last rows:
              Date       Station Observation Time (UTC)  Temperature (C)
0      2023-01-01     birzu-ams    2023-01-01 00:00:00              8.7
1      2023-01-01     birzu-ams    2023-01-01 01:00:00              9.3
2      2023-01-01     birzu-ams    2023-01-01 02:00:00              9.6
3      2023-01-01     birzu-ams    2023-01-01 03:00:00              9.8
4      2023-01-01     birzu-ams    2023-01-01 04:00:00             10.1
...           ...           ...                    ...              ...
13387  2023-01-31  vilniaus-ams    2023-01-31 19:00:00              0.0
13388  2023-01-31  vilniaus-ams    2023-01-31 20:00:00             -0.4
13389  2023-01-31  vilniaus-ams    2023-01-31 21:00:00             -0.6
13390  2023-01-31  vilniaus-ams    2023-01-31 22:00:00             -0.5
13391  2

Converting time and calculating average temperatur

In [5]:
all_data['Observation Time (UTC)'] = pd.to_datetime(all_data['Observation Time (UTC)'])

avg_temp_per_day_per_station = all_data.groupby(['Date', 'Station'])['Temperature (C)'].mean().reset_index()

print(avg_temp_per_day_per_station)

           Date        Station  Temperature (C)
0    2023-01-01      birzu-ams         8.758333
1    2023-01-01   dotnuvos-ams        10.250000
2    2023-01-01     duksto-ams         8.933333
3    2023-01-01      kauno-ams        11.362500
4    2023-01-01  klaipedos-ams         7.200000
..          ...            ...              ...
553  2023-01-31     telsiu-ams         1.245833
554  2023-01-31   ukmerges-ams         1.395833
555  2023-01-31     utenos-ams         1.133333
556  2023-01-31    varenos-ams         1.304167
557  2023-01-31   vilniaus-ams         0.808333

[558 rows x 3 columns]


In [11]:
avg_temp_per_day_LT = avg_temp_per_day_per_station.groupby(['Date'])['Temperature (C)'].mean().reset_index()

print(avg_temp_per_day_LT)

          Date  Temperature (C)
0   2023-01-01         9.967824
1   2023-01-02         6.658102
2   2023-01-03         4.427315
3   2023-01-04         0.704861
4   2023-01-05        -0.907407
5   2023-01-06       -11.771991
6   2023-01-07        -9.221528
7   2023-01-08        -3.150926
8   2023-01-09        -1.322685
9   2023-01-10         1.238657
10  2023-01-11         0.332176
11  2023-01-12         2.620370
12  2023-01-13         4.544907
13  2023-01-14         4.621759
14  2023-01-15         3.669676
15  2023-01-16         2.640046
16  2023-01-17         2.100000
17  2023-01-18         1.528241
18  2023-01-19         0.566667
19  2023-01-20        -0.924074
20  2023-01-21        -0.741204
21  2023-01-22        -1.314815
22  2023-01-23        -1.917130
23  2023-01-24        -2.174769
24  2023-01-25        -0.694907
25  2023-01-26        -1.203009
26  2023-01-27        -0.644213
27  2023-01-28        -1.733102
28  2023-01-29         0.259722
29  2023-01-30         1.296991
30  2023

Saving aggregated data

In [14]:
agg_stations_file_name = f'avg_temp_per_day_per_station_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}.csv'
avg_temp_per_day_per_station.to_csv(agg_stations_file_name, index=False)

print(f'Data saved to {agg_stations_file_name}')

Data saved to avg_temp_per_day_per_station_20230101_20230131.csv


In [15]:
agg_LT_file_name = f'avg_temp_per_day_LT_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}.csv'
avg_temp_per_day_LT.to_csv(agg_LT_file_name, index=False)

print(f'Data saved to {agg_LT_file_name}')

Data saved to avg_temp_per_day_LT_20230101_20230131.csv
